# Load dependencies

In [1]:
import sys
import os
import requests
import subprocess
import shutil
from logging import getLogger, StreamHandler, INFO


logger = getLogger(__name__)
logger.addHandler(StreamHandler())
logger.setLevel(INFO)


def install(
        chunk_size=4096,
        file_name="Miniconda3-latest-Linux-x86_64.sh",
        url_base="https://repo.continuum.io/miniconda/",
        conda_path=os.path.expanduser(os.path.join("~", "miniconda")),
        rdkit_version=None,
        add_python_path=True,
        force=False):
    """install rdkit from miniconda
    ```
    import rdkit_installer
    rdkit_installer.install()
    ```
    """

    python_path = os.path.join(
        conda_path,
        "lib",
        "python{0}.{1}".format(*sys.version_info),
        "site-packages",
    )

    if add_python_path and python_path not in sys.path:
        logger.info("add {} to PYTHONPATH".format(python_path))
        sys.path.append(python_path)

    if os.path.isdir(os.path.join(python_path, "rdkit")):
        logger.info("rdkit is already installed")
        if not force:
            return

        logger.info("force re-install")

    url = url_base + file_name
    python_version = "{0}.{1}.{2}".format(*sys.version_info)

    logger.info("python version: {}".format(python_version))

    if os.path.isdir(conda_path):
        logger.warning("remove current miniconda")
        shutil.rmtree(conda_path)
    elif os.path.isfile(conda_path):
        logger.warning("remove {}".format(conda_path))
        os.remove(conda_path)

    logger.info('fetching installer from {}'.format(url))
    res = requests.get(url, stream=True)
    res.raise_for_status()
    with open(file_name, 'wb') as f:
        for chunk in res.iter_content(chunk_size):
            f.write(chunk)
    logger.info('done')

    logger.info('installing miniconda to {}'.format(conda_path))
    subprocess.check_call(["bash", file_name, "-b", "-p", conda_path])
    logger.info('done')

    logger.info("installing rdkit")
    subprocess.check_call([
        os.path.join(conda_path, "bin", "conda"),
        "install",
        "--yes",
        "-c", "rdkit",
        "python=={}".format(python_version),
        "rdkit" if rdkit_version is None else "rdkit=={}".format(rdkit_version)])
    logger.info("done")

    import rdkit
    logger.info("rdkit-{} installation finished!".format(rdkit.__version__))


if __name__ == "__main__":
    install()

add /root/miniconda/lib/python3.6/site-packages to PYTHONPATH
python version: 3.6.9
fetching installer from https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
done
installing miniconda to /root/miniconda
done
installing rdkit
done
rdkit-2020.09.1 installation finished!


In [2]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [3]:
!cp -r "drive/My Drive/deepsiba_tf2/NGF" /content
!cp -r "drive/My Drive/deepsiba_tf2/NGF_layers" /content
!cp -r "drive/My Drive/deepsiba_tf2/utility" /content
!cp -r "drive/My Drive/deepsiba_tf2/utils" /content
!cp -r "drive/My Drive/deepsiba_tf2/data" /content
!cp "drive/My Drive/deepsiba_tf2/deepSIBA_model.py" /content

In [4]:
from __future__ import division, print_function
import numpy as np
from numpy import inf, ndarray
import pandas as pd
import tensorflow as tf
import os
import random
import keras
import sklearn
import re
from keras import optimizers, losses, regularizers
import keras.backend as K
from keras.models import model_from_json, load_model, Model
from tempfile import TemporaryFile
from keras import layers
from keras.callbacks import History, ReduceLROnPlateau
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout, Layer
from keras.initializers import glorot_normal
from keras.regularizers import l2
from functools import partial
from multiprocessing import cpu_count, Pool
from keras.utils.generic_utils import Progbar
from copy import deepcopy
from NGF.utils import filter_func_args, mol_shapes_to_dims
import NGF.utils
import NGF_layers.features
import NGF_layers.graph_layers
from NGF_layers.features import one_of_k_encoding, one_of_k_encoding_unk, atom_features, bond_features, num_atom_features, num_bond_features, padaxis, tensorise_smiles, concat_mol_tensors
from NGF_layers.graph_layers import temporal_padding, neighbour_lookup, NeuralGraphHidden, NeuralGraphOutput
from math import ceil
from sklearn.metrics import mean_squared_error
from utility.gaussian import GaussianLayer, custom_loss, ConGaussianLayer
from utility.evaluator import r_square, get_cindex, pearson_r,custom_mse, mse_sliced, model_evaluate
from utility.Generator import train_generator,preds_generator
from deepSIBA_model import enc_graph, siamese_model, deepsiba_model
from pathlib import Path
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

In [5]:
print(tf.__version__)

2.3.0


In [6]:
# Check available GPU devices.
print("The following GPU devices are available: %s" % tf.test.gpu_device_name())

The following GPU devices are available: /device:GPU:0


# Load train and model parameters

In [29]:
#model_params
model_params = {
    "max_atoms" : int(60), "num_atom_features" : int(62), "max_degree" : int(5), "num_bond_features" : int(6),
    "graph_conv_width" : [128,128,128], "conv1d_filters" : int(128), "conv1d_size" : int(29), "dropout_encoder" : 0.25,
    "conv1d_filters_dist" : [128,128], "conv1d_size_dist" : [17,1], "dropout_dist" : 0.25, "pool_size" : int(4),
    "dense_size" : [256,128,128], "l2reg" : 0.01, "dist_thresh" : 0.2, "lr" : 0.001 ,"ConGauss": False
}

In [8]:
train_params = {
    "cell_line" : "a375", "split" : "train_test_split", "number_folds" : [0],
    "output_dir" : "results",
    "batch_size" : int(128), "epochs" : int(20), 
    "N_ensemble" : int(1), "nmodel_start" : int(0), "prec_threshold" : 0.2,
    "Pre_training" : False,
    "Pre_trained_cell_dir" : '',
    "pattern_to_load" : 'siam_no_augment_',
    "model_id_to_load" : "20",
    "test_value_norm" : True,
    "predict_batch_size":int(2048)
}

# Load data

In [9]:
get_all = []
if train_params["split"] == "train_test_split":
  outer_loop = train_params["number_folds"]
elif train_params["split"] == "5_fold_cv_split":
  outer_loop = train_params["number_folds"]
elif train_params["split"] == "alldata":
  outer_loop = train_params["number_folds"]
#Load unique smiles and tensorize them
smiles = pd.read_csv("data/" + train_params["cell_line"] + "/" + train_params["cell_line"] + "q1smiles.csv", index_col=0)
X_atoms, X_bonds, X_edges = tensorise_smiles(smiles.x, model_params["max_degree"], model_params["max_atoms"])
smiles=list(smiles['x'])

In [10]:
df = pd.read_csv("data/" + train_params["cell_line"] + "/" + "train_test_split/" + "train.csv",index_col=0).reset_index(drop=True)
df_cold = pd.read_csv("data/" + train_params["cell_line"] + "/" + "train_test_split/" + "test.csv",index_col=0).reset_index(drop=True)
smiles_cold = list(set(list(df_cold['rdkit.x'])+list(df_cold['rdkit.y'])))
X_atoms_cold, X_bonds_cold, X_edges_cold = tensorise_smiles(smiles_cold,  model_params["max_degree"], model_params["max_atoms"])
#X_atoms_cold=X_atoms_cold.astype('float64')
#X_bonds_cold=X_bonds_cold.astype('float64')
#X_edges_cold=X_edges_cold.astype('int64')
if train_params["test_value_norm"]:
  Y_cold = df_cold.value
else:
  Y_cold = df_cold.value
  Y_cold = Y_cold/2

In [11]:
i=0
Path(train_params["output_dir"] + "/" + "fold_%s/models"%i).mkdir(parents=True, exist_ok=True)
cold_preds_mus = []
cold_preds_sigmas = []
n = train_params["nmodel_start"]

# Define,Compile,Train model

In [46]:
deepsiba = deepsiba_model(model_params)

In [31]:
#atoms0_1 = Input(name='atom_inputs_1', shape=(model_params["max_atoms"], model_params["num_atom_features"]),dtype = 'float32')
#bonds_1 = Input(name='bond_inputs_1', shape=(model_params["max_atoms"], model_params["max_degree"], model_params["num_bond_features"]),dtype = 'float32')
#edges_1 = Input(name='edge_inputs_1', shape=(model_params["max_atoms"], model_params["max_degree"]), dtype='int32')

#atoms0_2 = Input(name='atom_inputs_2',shape=(model_params["max_atoms"], model_params["num_atom_features"]),dtype = 'float32')
#bonds_2 = Input(name='bond_inputs_2',shape=(model_params["max_atoms"], model_params["max_degree"], model_params["num_bond_features"]),dtype = 'float32')
#edges_2 = Input(name='edge_inputs_2', shape=(model_params["max_atoms"], model_params["max_degree"]), dtype='int32')

In [32]:
#fake_out=deepsiba(atoms0_1,bonds_1,edges_1,atoms0_2,bonds_2,edges_2)

In [33]:
deepsiba.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
atom_inputs_1 (InputLayer)      [(None, 60, 62)]     0                                            
__________________________________________________________________________________________________
bond_inputs_1 (InputLayer)      [(None, 60, 5, 6)]   0                                            
__________________________________________________________________________________________________
edge_inputs_1 (InputLayer)      [(None, 60, 5)]      0                                            
__________________________________________________________________________________________________
atom_inputs_2 (InputLayer)      [(None, 60, 62)]     0                                            
_______________________________________________________________________________________

In [34]:
#for i, w in enumerate(deepsiba.weights): print(i, w.name)

In [35]:
if train_params["Pre_training"]:
  weight_path=train_params["Pre_trained_cell_dir"]+"/"+train_params["pattern_to_load"]+train_params["model_id_to_load"]+".h5"
  deepsiba.load_weights(weight_path)

In [36]:
rlr = ReduceLROnPlateau(monitor='loss', factor=0.5,patience=3, min_lr=0.00001, verbose=1, min_delta=1e-5)
term=keras.callbacks.TerminateOnNaN()
bs = train_params["batch_size"]
NUM_EPOCHS = train_params["epochs"]
df = df.sample(frac=1).reset_index(drop=True)
NUM_TRAIN = len(df)
NUM_STEPS=ceil(NUM_TRAIN/bs)
trainGen=train_generator(bs,df,smiles,X_atoms, X_bonds, X_edges)
check = NUM_EPOCHS-1

In [37]:
#@tf.function
#def train_step(atom_1,bond_1,edge_1,atom_2,bond_2,edge_2,y):
#  with tf.GradientTape() as tape:
#    # training=True is only needed if there are layers with different
#    # behavior during training versus inference (e.g. Dropout)
#    predictions = deepsiba(atom_1,bond_1,edge_1,atom_2,bond_2,edge_2, training=True)
#    loss = custom_loss(y, predictions)
#  gradients = tape.gradient(loss, deepsiba.trainable_variables)
#  adam.apply_gradients(zip(gradients, deepsiba.trainable_variables))
#  train_loss(loss)

In [38]:
#lp=round(NUM_STEPS*0.1)
#lp

In [39]:
#cn=0
#for epoch in range(NUM_EPOCHS):
#  train_loss.reset_states()
#  #train_accuracy.reset_states()
#  for step in range(NUM_STEPS):
#    Xatom_1,Xbond_1,Xedge_1,Xatom_2,Xbond_2,Xedge_2,y = list(next(trainGen))
#    train_step(Xatom_1,Xbond_1,Xedge_1,Xatom_2,Xbond_2,Xedge_2,y)
#    if (step%lp==0):
#      cn+=1
#      if (epoch==0):
#        sys.stdout.write("\r"+"Epoch "+str(epoch + 1)+": "+"Steps: "+str(step+1)+"/"+str(NUM_STEPS)+": "f'Loss: {train_loss.result()}')
#      else:
#        if (cn==1):
#          print("Epoch "+str(epoch + 1)+": "+"Steps: "+str(step+1)+"/"+str(NUM_STEPS)+": "f'Loss: {train_loss.result()}')
#        else:
#          sys.stdout.write("\r"+"Epoch "+str(epoch + 1)+": "+"Steps: "+str(step+1)+"/"+str(NUM_STEPS)+": "f'Loss: {train_loss.result()}')
#  cn=0

In [40]:
if train_params["split"] != "alldata":
  Path(train_params["output_dir"] + "/" + "fold_%s/cold/mu"%i).mkdir(parents=True, exist_ok=True)
  Path(train_params["output_dir"] + "/" + "fold_%s/cold/sigma"%i).mkdir(parents=True, exist_ok=True)
  Path(train_params["output_dir"] + "/" + "fold_%s/performance"%i).mkdir(parents=True, exist_ok=True)

In [41]:
history = deepsiba.fit(trainGen,
                       steps_per_epoch= ceil(NUM_TRAIN/bs),
                       epochs=NUM_EPOCHS,
                       verbose = 1,
                       shuffle = True,
                       callbacks= [term, rlr])

Epoch 1/10
1367/1367 [==============================] - 91s 66ms/step - loss: -0.6091 - custom_mse: 0.0238 - get_cindex: 0.6086 - r_square: -0.9413 - pearson_r: 0.2691 - mse_similars: 0.0796
Epoch 2/10
1367/1367 [==============================] - 90s 66ms/step - loss: -1.7985 - custom_mse: 0.0103 - get_cindex: 0.7102 - r_square: 0.1522 - pearson_r: 0.5052 - mse_similars: nan
Epoch 3/10
1367/1367 [==============================] - 90s 66ms/step - loss: -1.9330 - custom_mse: 0.0096 - get_cindex: 0.7388 - r_square: 0.2107 - pearson_r: 0.5546 - mse_similars: nan
Epoch 4/10
1367/1367 [==============================] - 90s 66ms/step - loss: -2.0469 - custom_mse: 0.0077 - get_cindex: 0.7780 - r_square: 0.3660 - pearson_r: 0.6595 - mse_similars: nan
Epoch 5/10
1367/1367 [==============================] - 91s 66ms/step - loss: -2.1488 - custom_mse: 0.0057 - get_cindex: 0.8024 - r_square: 0.5265 - pearson_r: 0.7505 - mse_similars: nan
Epoch 6/10
1367/1367 [==============================] - 90s 6

In [24]:
if history.history["r_square"][len(history.history["r_square"])-1] < 0.7:
  history = deepsiba.fit(trainGen,
                         steps_per_epoch= ceil(NUM_TRAIN/bs),
                         epochs=10,
                         verbose = 1,
                         shuffle = True,
                         callbacks= [term, rlr])

In [42]:
deepsiba.save_weights('my_model2.h5')

In [43]:
### kai predict generator ###
if train_params["split"] != "alldata":
  #gaussian = keras.Model(deepsiba.inputs, deepsiba.get_layer('main_output').output)
  pr_steps=ceil(len(df_cold)/train_params["predict_batch_size"])
  PredGen=preds_generator(train_params["predict_batch_size"],df_cold,smiles_cold,X_atoms_cold, X_bonds_cold, X_edges_cold,deepsiba)
  y_pred1=[]
  y_pred2=[]
  for g in range(pr_steps):
    cold_pred=list(next(PredGen))
    y_pred1=y_pred1+list(cold_pred[0])
    y_pred2=y_pred2+list(cold_pred[1])
y_pred1=np.array(y_pred1)
y_pred2=np.array(y_pred2)
if (len(y_pred1[np.where(y_pred1 <= train_params["prec_threshold"])])>0):
  get = model_evaluate(y_pred1,Y_cold,train_params["prec_threshold"],df_cold)
  get.to_csv(train_params["output_dir"] + "/" + "fold_%s/performance/"%i + "model_%s.csv"%n)
cold_preds_mus.append(y_pred1)
np.save(train_params["output_dir"] + "/" + "fold_%s/cold/mu/"%i + "cold_mu_%s.npy"%n, y_pred1)
cold_preds_sigmas.append(y_pred2)
np.save(train_params["output_dir"] + "/" + "fold_%s/cold/sigma/"%i + "cold_sigma_%s.npy"%n, y_pred2)

In [44]:
if train_params["split"] != "alldata":
  mu_star=np.mean(cold_preds_mus,axis=0)
  sigma_star = np.sqrt(np.mean(cold_preds_sigmas + np.square(cold_preds_mus), axis = 0) - np.square(mu_star))
  cv_star = sigma_star/mu_star
  if (len(mu_star[np.where(mu_star <= train_params["prec_threshold"])])>0):
      get_fold = model_evaluate(mu_star,Y_cold,train_params["prec_threshold"],df_cold)
      get_fold.to_csv(train_params["output_dir"] + "/" + "fold_%s/ensemble_performance.csv"%i)
      get_all.append(get_fold)
  df_cold['mu'] = mu_star
  df_cold['cv'] = cv_star
  df_cold.to_csv(train_params["output_dir"] + "/" + "fold_%s/ensemble_preds_dataframe.csv"%i)

In [45]:
df_cold

,X,Var1,Var2,value,sig_id.x,pert_iname.x,quality.x,rdkit.x,pert_dose.x,pert_time.x,sig_id.y,pert_iname.y,quality.y,rdkit.y,pert_dose.y,pert_time.y,V1,sigma_star,dist,pair,mu,cv
0,91,BRD-A51714012,BRD-A54880345,0.156852,CPC004_A375_6H:BRD-A51714012-001-03-1:10,venlafaxine,1,COc1ccc(C(CN(C)C)C2(O)CCCCC2)cc1,10.0000,6,CPC004_A375_6H:BRD-A54880345-001-11-8:10,etomidate,1,CCOC(=O)c1cncn1C(C)c1ccccc1,10.0000,6,0.323492,0.046807,0.841270,COc1ccc(C(CN(C)C)C2(O)CCCCC2)cc1 and CCOC(=O)c...,0.314323,0.121557
1,104,BRD-A51714012,BRD-A62184259,0.327250,CPC004_A375_6H:BRD-A51714012-001-03-1:10,venlafaxine,1,COc1ccc(C(CN(C)C)C2(O)CCCCC2)cc1,10.0000,6,CPC004_A375_6H:BRD-A62184259-001-02-8:10,cycloheximide,1,CC1CC(C)C(=O)C(C(O)CC2CC(=O)NC(=O)C2)C1,10.0000,6,0.299491,0.045602,0.888889,COc1ccc(C(CN(C)C)C2(O)CCCCC2)cc1 and CC1CC(C)C...,0.284529,0.127238
2,118,BRD-A51714012,BRD-A83326220,0.318128,CPC004_A375_6H:BRD-A51714012-001-03-1:10,venlafaxine,1,COc1ccc(C(CN(C)C)C2(O)CCCCC2)cc1,10.0000,6,CPC004_A375_6H:BRD-A83326220-001-04-2:10,brazilin,1,Oc1ccc2c(c1)OCC1(O)Cc3cc(O)c(O)cc3C21,10.0000,6,0.243868,0.049710,0.857143,COc1ccc(C(CN(C)C)C2(O)CCCCC2)cc1 and Oc1ccc2c(...,0.245082,0.138234
3,133,BRD-A51714012,BRD-A93236127,0.315317,CPC004_A375_6H:BRD-A51714012-001-03-1:10,venlafaxine,1,COc1ccc(C(CN(C)C)C2(O)CCCCC2)cc1,10.0000,6,CPC004_A375_6H:BRD-A93236127-001-03-7:10,digitoxin,1,C[C@H]1O[C@@H](O[C@H]2[C@@H](O)C[C@H](O[C@H]3[...,10.0000,6,0.285898,0.039665,0.901099,COc1ccc(C(CN(C)C)C2(O)CCCCC2)cc1 and C[C@H]1O[...,0.267207,0.131542
4,149,BRD-A51714012,BRD-A93424738,0.179933,CPC004_A375_6H:BRD-A51714012-001-03-1:10,venlafaxine,1,COc1ccc(C(CN(C)C)C2(O)CCCCC2)cc1,10.0000,6,CPC004_A375_6H:BRD-A93424738-001-03-0:10,dexamethasone-acetate,1,CC(=O)OCC(=O)[C@@]1(O)[C@H](C)CC2C3CCC4=CC(=O)...,10.0000,6,0.352832,0.092941,0.880952,COc1ccc(C(CN(C)C)C2(O)CCCCC2)cc1 and CC(=O)OCC...,0.357562,0.135550
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45579,224780,BRD-K48222931,BRD-K96041907,0.378380,DOSBIO001_A375_24H:BRD-K48222931:10.2274,BRD-K48222931,1,C[C@@H](CO)N1C[C@@H](C)[C@H](CN(C)Cc2ccccc2)OC...,10.2274,24,DOSBIO001_A375_24H:BRD-K96041907:10.2154,BRD-K96041907,1,CCCNC(=O)N(C)C[C@H]1N[C@@H](CO)[C@@H]1c1ccc(C2...,10.2154,24,0.298532,0.042265,0.831683,C[C@@H](CO)N1C[C@@H](C)[C@H](CN(C)Cc2ccccc2)OC...,0.289723,0.133837
45580,224781,BRD-K62242359,BRD-K96041907,0.601630,DOSBIO001_A375_24H:BRD-K62242359:10.5421,BRD-K62242359,1,CC(C)NC(=O)Nc1ccc2c(c1)C(=O)N([C@@H](C)CO)C[C@...,10.5421,24,DOSBIO001_A375_24H:BRD-K96041907:10.2154,BRD-K96041907,1,CCCNC(=O)N(C)C[C@H]1N[C@@H](CO)[C@@H]1c1ccc(C2...,10.2154,24,0.297804,0.051306,0.780000,CC(C)NC(=O)Nc1ccc2c(c1)C(=O)N([C@@H](C)CO)C[C@...,0.293257,0.153426
45581,224783,BRD-K88822846,BRD-K96041907,0.475502,DOSBIO001_A375_24H:BRD-K88822846:10.1074,BRD-K88822846,1,C[C@@H]1CN([C@H](C)CO)C(=O)c2cc(NS(C)(=O)=O)cc...,10.1074,24,DOSBIO001_A375_24H:BRD-K96041907:10.2154,BRD-K96041907,1,CCCNC(=O)N(C)C[C@H]1N[C@@H](CO)[C@@H]1c1ccc(C2...,10.2154,24,0.298268,0.050708,0.817308,C[C@@H]1CN([C@H](C)CO)C(=O)c2cc(NS(C)(=O)=O)cc...,0.294517,0.172014
45582,224784,BRD-K89961851,BRD-K96041907,0.435324,DOSBIO001_A375_24H:BRD-K89961851:10.2368,BRD-K89961851,1,N#C[C@H]1[C@@H](c2ccc(C3=CCCCC3)cc2)[C@H](CO)N...,10.2368,24,DOSBIO001_A375_24H:BRD-K96041907:10.2154,BRD-K96041907,1,CCCNC(=O)N(C)C[C@H]1N[C@@H](CO)[C@@H]1c1ccc(C2...,10.2154,24,0.279089,0.045356,0.645570,N#C[C@H]1[C@@H](c2ccc(C3=CCCCC3)cc2)[C@H](CO)N...,0.273105,0.153280
